In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score,mean_squared_error,precision_score

import warnings
warnings.filterwarnings('always')

In [2]:
data = pd.read_csv(r"C:\Users\Derick\Desktop\Dataset\MSLMAIN.csv")
data.head()

,Team,Date,Time,Venue,Opponent,Goal Scored,Goal Conceded,Results,Stadium,FoulF,...,CornerF,CornerA,OffF,OffA,ShotF,ShotA,ShotOffF,ShotOffA,ShotOnF,ShotOnA
0,JOHOR DARUL TA'ZIM,5/4/21,10:00 PM,A,KEDAH DARUL AMAN FC,1,0,W,"STADIUM DARUL AMAN, ALOR SETAR",25,...,6,1,1,2,13,4,9,3,4,1
1,JOHOR DARUL TA'ZIM,4/30/21,10:00 PM,A,KUALA LUMPUR CITY FC,1,1,D,"STADIUM BOLASEPAK KUALA LUMPUR,CHERAS",12,...,10,5,1,1,16,7,14,4,2,3
2,JOHOR DARUL TA'ZIM,8/3/21,9:00 PM,A,MELAKA UNITED FC,1,0,W,"STADIUM HANG JEBAT, MELAKA",28,...,2,4,2,0,8,4,6,3,2,1
3,JOHOR DARUL TA'ZIM,5/8/21,10:00 PM,A,PENANG FC,3,0,W,"STADIUM BANDARAYA, PULAU PINANG",22,...,4,3,3,2,7,4,1,4,6,0
4,JOHOR DARUL TA'ZIM,3/21/21,9:00 PM,A,PERAK FC,2,2,D,"STADIUM PERAK, IPOH",19,...,18,1,2,0,26,6,13,4,13,2


In [3]:
data.shape

(828, 21)

In [4]:
data["Team"].value_counts()

Team
SELANGOR FC             67
JOHOR DARUL TA'ZIM      66
SABAH FC                66
KUALA LUMPUR CITY FC    66
PENANG FC               66
KEDAH DARUL AMAN FC     66
TERENGGANU FC           65
SRI PAHANG FC           65
PJ CITY FC              44
PERAK FC                44
MELAKA UNITED FC        44
NEGERI SEMBILAN FC      44
UITM FC                 22
SARAWAK UNITED FC       22
KELANTAN FC             21
KELANTAN UNITED FC      21
PDRM FC                 20
KUCHING CITY FC         19
Name: count, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Team           828 non-null    object
 1   Date           828 non-null    object
 2   Time           828 non-null    object
 3   Venue          828 non-null    object
 4   Opponent       828 non-null    object
 5   Goal Scored    828 non-null    int64 
 6   Goal Conceded  828 non-null    int64 
 7   Results        828 non-null    object
 8   Stadium        828 non-null    object
 9   FoulF          828 non-null    int64 
 10  FoulA          828 non-null    int64 
 11  CornerF        828 non-null    int64 
 12  CornerA        828 non-null    int64 
 13  OffF           828 non-null    int64 
 14  OffA           828 non-null    int64 
 15  ShotF          828 non-null    int64 
 16  ShotA          828 non-null    int64 
 17  ShotOffF       828 non-null    int64 
 18  ShotOffA       828 non-null   

### Data Preperation

In [6]:
data["Date"] = pd.to_datetime(data["Date"])

C:\Users\Derick\AppData\Local\Temp\ipykernel_2112\2487158108.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"])


In [7]:
data["Venue_Code"] = data["Venue"].astype("category").cat.codes

In [8]:
data["Team_code"] = data["Team"].astype("category").cat.codes

In [9]:
data["opp_code"] = data["Opponent"].astype("category").cat.codes

In [10]:
data["hour"] = data["Time"].str.replace(":.+", "", regex=True).astype("int")

In [11]:
data["day_code"] = data["Date"].dt.dayofweek

In [12]:
data["target"] = (data["Results"] == "W").astype("int")

### EDA

### Model Training

In [13]:
train = data[data["Date"] < '2022-01-01']

In [14]:
test = data[data["Date"] > '2022-01-01']

In [15]:
predictors1 = ["Venue_Code", "opp_code", "hour", "day_code"]

SVM

In [16]:
sv = SVC(kernel='rbf', C=3, probability=True)
sv1 = SVC(kernel='rbf', C=3, probability=True)

In [17]:
sv.fit(train[predictors1], train["target"])

SVC(C=3, probability=True)

In [18]:
svpreds = sv.predict(test[predictors1])

In [19]:
svmacc = accuracy_score(test["target"], svpreds)
svmacc

0.5957446808510638

Random Forest

In [20]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf1 = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [21]:
rf.fit(train[predictors1], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [22]:
preds = rf.predict(test[predictors1])

In [23]:
rfacc = accuracy_score(test["target"], preds)
rfacc

0.5939716312056738

In [24]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [25]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,306,30
1,199,29


In [26]:
grouped_matches = data.groupby("Team")

In [27]:
group = grouped_matches.get_group("PERAK FC")

In [28]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3,closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [29]:
cols = ["Goal Scored","Goal Conceded","ShotF","ShotOnF","CornerF","CornerA","OffF","OffA","FoulF","FoulA"]
predictors = ["Team_code","Venue_Code", "opp_code", "hour", "day_code"]
new_cols =[f"{c}_rolling" for c in cols]

In [30]:
new_cols

['Goal Scored_rolling',
 'Goal Conceded_rolling',
 'ShotF_rolling',
 'ShotOnF_rolling',
 'CornerF_rolling',
 'CornerA_rolling',
 'OffF_rolling',
 'OffA_rolling',
 'FoulF_rolling',
 'FoulA_rolling']

In [31]:
rolling_averages(group, cols, new_cols)

,Team,Date,Time,Venue,Opponent,Goal Scored,Goal Conceded,Results,Stadium,FoulF,...,Goal Scored_rolling,Goal Conceded_rolling,ShotF_rolling,ShotOnF_rolling,CornerF_rolling,CornerA_rolling,OffF_rolling,OffA_rolling,FoulF_rolling,FoulA_rolling
57,PERAK FC,2021-03-17,9:00 PM,A,KUALA LUMPUR CITY FC,0,3,L,"STADIUM BOLASEPAK KUALA LUMPUR,CHERAS",8,...,0.666667,0.666667,10.333333,5.333333,2.666667,4.666667,0.666667,4.333333,12.666667,14.666667
319,PERAK FC,2021-03-21,9:00 PM,H,JOHOR DARUL TA'ZIM,2,2,D,"STADIUM PERAK, IPOH",18,...,0.666667,1.666667,9.333333,4.000000,2.666667,4.333333,0.000000,5.333333,10.666667,14.333333
59,PERAK FC,2021-04-03,9:00 PM,A,PENANG FC,1,2,L,"STADIUM BANDARAYA, PULAU PINANG",17,...,0.666667,2.333333,9.333333,3.666667,2.666667,8.333333,0.000000,4.666667,11.666667,16.333333
329,PERAK FC,2021-04-07,9:00 PM,H,UITM FC,3,2,W,"STADIUM PERAK, IPOH",9,...,1.000000,2.333333,9.666667,3.000000,2.666667,7.666667,1.000000,3.333333,14.333333,17.333333
62,PERAK FC,2021-04-10,9:00 PM,A,SELANGOR FC,1,3,L,STADIUM MAJLIS BANDARAYA PETALING JAYA,12,...,2.000000,2.000000,14.333333,5.666667,5.000000,8.000000,1.333333,1.000000,14.666667,15.000000
320,PERAK FC,2021-04-17,10:00 PM,H,KEDAH DARUL AMAN FC,1,1,D,"STADIUM PERAK, IPOH",13,...,1.666667,2.333333,15.666667,6.666667,5.333333,4.333333,2.666667,0.666667,12.666667,13.666667
322,PERAK FC,2021-04-25,10:00 PM,H,MELAKA UNITED FC,0,1,L,"STADIUM PERAK, IPOH",15,...,1.666667,2.000000,15.666667,7.333333,5.666667,3.666667,2.666667,1.333333,11.333333,12.000000
61,PERAK FC,2021-05-02,9:15 PM,A,SABAH FC,1,2,L,"STADIUM LIKAS, KOTA KINABALU",16,...,0.666667,1.666667,10.333333,4.666667,4.000000,4.666667,2.666667,1.666667,13.333333,15.333333
60,PERAK FC,2021-05-05,10:00 PM,A,PJ CITY FC,1,0,W,STADIUM MAJLIS BANDARAYA PETALING JAYA,18,...,0.666667,1.333333,9.000000,4.000000,4.000000,3.666667,1.333333,2.333333,14.666667,16.666667
327,PERAK FC,2021-05-09,10:00 PM,H,SRI PAHANG FC,2,3,L,"STADIUM PERAK, IPOH",18,...,0.666667,1.000000,6.666667,2.333333,4.000000,4.333333,0.666667,2.666667,16.333333,19.333333


In [32]:
matches_rolling = data.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('Team')
matches_rolling.index = range(matches_rolling.shape[0])

In [33]:
matches_rolling.to_csv(r"C:\Users\Derick\Desktop\Dataset\MSLROLLALL.csv", index=True)

Retrain Model

In [34]:
allCols = predictors + new_cols

In [35]:
allCols

['Team_code',
 'Venue_Code',
 'opp_code',
 'hour',
 'day_code',
 'Goal Scored_rolling',
 'Goal Conceded_rolling',
 'ShotF_rolling',
 'ShotOnF_rolling',
 'CornerF_rolling',
 'CornerA_rolling',
 'OffF_rolling',
 'OffA_rolling',
 'FoulF_rolling',
 'FoulA_rolling']

Split data for train and test

In [36]:
train = matches_rolling[matches_rolling["Date"] < '2023-01-01']
test = matches_rolling[matches_rolling["Date"] > '2023-01-01']

Retrain Random Forest Model with rolling averages

In [37]:
rf1.fit(train[allCols], train["target"])
preds1 = rf1.predict(test[allCols])
acc = accuracy_score(test["target"], preds1)
print(f'Accuracy: {acc * 100:.2f}%')

Accuracy: 69.79%


Retrain SVM Model with rolling averages

In [38]:
sv1.fit(train[allCols], train["target"])
preds1 = sv1.predict(test[allCols])
acc = accuracy_score(test["target"], preds1)
acc

0.6701388888888888

Grid Search

In [40]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')
svm_classifier = SVC()

train_target = train["target"]
test_target = test["target"]

train_features = train[allCols]
test_features = test[allCols]

Random Forest Grid Search

In [41]:
# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [170],
    'max_depth': [5],
    'max_features': ['sqrt'],
    'criterion' :['entropy'],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit the model using grid search on the training data
grid_search.fit(train_features, train_target)

# Get the best parameters from grid search
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
best_rf_model = grid_search.best_estimator_
preds1 = best_rf_model.predict(test_features)

# Calculate accuracy
acc = accuracy_score(test_target, preds1)

# Print or use the best parameters and accuracy value as needed
print(f'Best Parameters: {best_params}')
print(f'Accuracy: {acc * 100:.2f}%')

Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 170}
Accuracy: 69.44%


SVC Grid Search

In [42]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit the model using grid search on the training data
grid_search.fit(train_features, train_target)

# Get the best parameters from grid search
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
best_svm_model = grid_search.best_estimator_
preds1 = best_svm_model.predict(test_features)

# Calculate accuracy
acc = accuracy_score(test_target, preds1)

# Print or use the best parameters and accuracy value as needed
print(f'Best Parameters: {best_params}')
print(f'Accuracy: {acc * 100:.2f}%')

Best Parameters: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
Accuracy: 61.11%


In [43]:
import joblib

In [44]:
best_rf_model = grid_search.best_estimator_
joblib.dump(best_rf_model, 'best_rf_model.joblib')

['best_rf_model.joblib']

In [45]:
joblib.dump(rf1, 'football_prediction_model1.joblib')

['football_prediction_model1.joblib']

In [46]:
joblib.dump(sv1, 'football_prediction_SVM.joblib')

['football_prediction_SVM.joblib']